<a href="https://colab.research.google.com/github/educatorsRlearners/hugging_face_course/blob/main/04_sharing_models_and_tokenizers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using pretrained models

The Model Hub makes it incredibly straightforward to selecet a pretrained model based on the task we want to achieve. 

Why is that important? 

Becasue, as Alan Maley once said, 

> The question isn't "We have this tech; what can we use it for?" The question is, "We have this problem. What tech can we use to solve it?" 

So, if we want to fill some masks in French, we can import ```camembert-base``` and use it in a pipeline.

In [13]:
pip install transformers sentencepiece 

     |████████████████████████████████| 1.2 MB 25.7 MB/s 


In [11]:
from transformers import  pipeline

checkpoint = 'camembert-base'
task = "fill-mask"

camembert_fill_mask = pipeline(task, checkpoint)

result = camembert_fill_mask("Le camembert est <mask> :) ")

for r in result:
  print(round(r['score'], 2), r["token_str"], r['sequence'], sep="---" )

0.49---délicieux---Le camembert est délicieux :)
0.11---excellent---Le camembert est excellent :)
0.03---succulent---Le camembert est succulent :)
0.03---meilleur---Le camembert est meilleur :)
0.03---parfait---Le camembert est parfait :)


Easy right? 

Just be careful to check and see that the checkpoint's head is suitable for the task. For instance, the ```camembert-base``` checkpoint has not been trained for sequence classification tasks. 

Finally, we can instantiate the checkpoint directly using the either the model architecture:

In [14]:
from transformers import CamembertTokenizer, CamembertForMaskedLM

tokenizer = CamembertTokenizer.from_pretrained(checkpoint)
moedl = CamembertForMaskedLM.from_pretrained(checkpoint)

But, if you want to be more flexible, then use the ```Auto*``` classes like this: 

In [15]:
from transformers import  AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForMaskedLM.from_pretrained(checkpoint)

# [Sharing pretrained models](https://huggingface.co/course/chapter4/3?fw=pt)